In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sentence_transformers.models import Pooling

pooling_layer = Pooling(768)
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained(
    "../../data/models/bert-base-finetuned-twitter16-tf/checkpoint-350/",
    output_hidden_states=True, num_labels=2)

model.to("cuda")
print("Loading done")

Loading done


In [2]:
def encode_text(text):
    inputs = tokenizer(text, return_tensors="pt")
    inputs.to("cuda")
    token_logits = model(inputs['input_ids'])

    embeddings = token_logits.hidden_states[-1]

    inputs = {
        'attention_mask': inputs['attention_mask'],
        'token_embeddings': embeddings,
        'cls_token_embeddings': None
    }
    result = pooling_layer.forward(inputs)
    result = result['sentence_embedding'].cpu().detach().numpy()

    return result

def encode_and_save(texts, savefile = None):
    if not savefile:
        print("Please define savefile")
        return
    
    vectors = []
    for text in texts:
        vectors.append(encode_text(text))
    
    vectors = np.array(vectors).reshape((len(texts), 768))
    
    with open(f"../../data/vectors/{savefile}.txt", "wb") as f:
        np.savetxt(f, vectors, delimiter=',', fmt='%s')

In [3]:
import pandas as pd
import numpy as np

data = pd.read_csv("../../data/twitter16-tf_dataset.csv", lineterminator="\n")
data.head()

tweet_id                                         tweet_text  \
0  656955120626880512  correct predictions in back to the future ii U...   
1  615689290706595840  .@whitehouse in rainbow colors for #scotusmarr...   
2  613404935003217920  cops bought the alleged church shooter burger ...   
3  614467824313106432     god put a rainbow over the white house 🌈 URL\r   
4  622891631293935616  #wakeupamerica🇺🇸 who needs a #gun registry whe...   

   label        tvt2  
0  False    training  
1   True    testting  
2  False  validation  
3   True  validation  
4  False    training

In [4]:
savefile = "Twitter16-TF_BERT_base_finetuned_vectors"
texts = data['tweet_text'].tolist()

encode_and_save(texts, savefile)

In [5]:
vectors = np.loadtxt(f"../../data/vectors/{savefile}.txt", delimiter=",")
vectors.shape

(412, 768)